# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
import math as math
from scipy import stats
import matplotlib.pyplot as plt
import re

### Selecionando a database

In [4]:
treino= pd.read_excel("Fortnite_treinamento_final.xlsx")
teste = pd.read_excel("Fortnite_teste_final.xlsx")

### Separando os relevantes do irrelevantes

In [6]:
treino_relevante = treino[treino.Relevância==1]
treino_irrelevante = treino[treino.Relevância ==0]

### Selecionando o conteúdo de cada database

Retiramos os tweets sob a coluna 'Treinamento'e inserimos em duas listas:

In [8]:
lista_tweets_irr= []
lista_tweets_rel=[]

def insere_db(data_rel,data_irr):
 
    l_rel=data_rel['Treinamento'].keys()
    l_irr=data_irr['Treinamento'].keys()

    for i in l_rel:
        lista_tweets_rel.append(data_rel['Treinamento'][i])
    for i in l_irr:
        lista_tweets_irr.append(data_irr['Treinamento'][i])
    
    
    

    return lista_tweets_rel, lista_tweets_irr


"Salvamos" as listas em duas variáveis diferentes, uma referindo aos relevantes e outra aos irrelevantes: 

In [9]:
relev,irrelev=insere_db(treino_relevante,treino_irrelevante)


Depois disso, formulamos uma função que separa cada palavra de uma frase e retira um conjunto de caracteres. Feito isso, coloca cada palavra em duas novas listas, uma para os relevantes e outra para os irrelevantes.

In [35]:
def separa(string):
    return re.split(",|\n|:|;|\(|\)|@| |!",string)


new_rel=[]
for v in relev:
    new_rel=new_rel+separa(v)
for v in new_rel:
    if v=='':
        new_rel.remove(v)
    if v=='/':
        new_rel.remove(v)
    if v== '\'':
        new_rel.remove(v)
    if v== ':':
        new_rel.remove(v)
    if v== ';':
        new_rel.remove(v)
    if v== '\n':
        new_rel.remove(v)
    
            
new_irr=[]
for v in irrelev:
    new_irr=new_irr+separa(v)
for v in new_irr:
    if v=='':
        new_irr.remove(v)
 
    

Agora, inserimos as palavras em dois novos dicionários, sendo que o valor de cada chave é o número de ocorrências de cada palavra:

In [12]:
dic_rel = {}
dic_irr = {}

#contando ocorrência de palavras
for i in new_rel:
    if i not in dic_rel:
        dic_rel[i]=1
        
    else:
        dic_rel[i]+=1

for i in new_irr:
    if i not in dic_irr:
        dic_irr[i]=1
        
    else:
        dic_irr[i]+=1
    

O próximo passo é criar uma lista que tem todas as palavras, sem repetição. Pela função "len" do python, encontramos o número de elementos da nova lista, e assim o número de palavras diferentes: 

In [16]:
#contando palavras sem repetição
lista_sem_rep = []
for i in new_rel:
    if i not in lista_sem_rep:
        lista_sem_rep.append(i)
for i in new_irr:
    if i not in lista_sem_rep:
        lista_sem_rep.append(i)

total = len(lista_sem_rep)
print(total)
    

1624


Da mesma forma, salvamos o número de tweets relevantes e irrelevantes em duas variáveis:

In [17]:
x = len(lista_tweets_rel)
y = len(lista_tweets_irr)



In [18]:
print(x,y)

251 49


Com esses dados, calculamos a probabilidade de um desses tweets ser irrelevante ou relevante:

In [19]:
prob_rel= x/300

prob_irr=1-prob_rel
print(prob_rel)

0.8366666666666667


Calculamos o número de palavras relevantes e irrelevantes, também contando as repetidas:

In [20]:
numero_pal_rel = len(new_rel)
numero_pal_irr = len(new_irr)

In [21]:
print(numero_pal_rel)

4578


In [22]:
print(numero_pal_irr)

953


Usando essas diversas variáveis, formulamos uma função que calcula a probabilidade de um certo tweet ser relevante:

In [63]:
#número de ocorrências da palavra mais 1, dividido pela soma do número de palavras relevantes/irrelevantes + número total de palavras sem repetição
def calcula_prob(tweet):
    palavras_tweet = separa(tweet)
    
    produto_rel= prob_rel
    produto_irr = prob_irr
    for i in palavras_tweet:
        if i in dic_rel:
            prob_tweet_rel =(dic_rel[i]+1)/(total+numero_pal_rel)
        else:
            prob_tweet_rel =1/(total+numero_pal_rel)
        if i in dic_irr:
            prob_tweet_irr =(dic_irr[i]+1)/(total+numero_pal_irr)
        else:
            prob_tweet_irr =1/(total+numero_pal_irr)
        
        produto_rel*=prob_tweet_rel
        produto_irr*= prob_tweet_irr
        k = 100*produto_rel/(produto_rel+produto_irr)
        h = 100*produto_irr/(produto_rel+produto_irr)
        z = k-h
#adicionamos tambem o quao relevante ele era baseado na diferenca de probabilidades        
    if z >=-100 and z<-50:
        print( 'extremamente irrelevante')
    elif z >=-50 and z<-10:
        print ('irrelevante')
    elif z >=-10 and z<10:
        print ('neutro')
    elif z>=10 and z<50:
        print ('relevante')
    elif z>=50 and z<100:
        print ('extremamente relevante')
    else:
        print ('ERRO')
            
            
    
    
    

In [64]:
print(calcula_prob('fortnite'))

extremamente relevante
None


___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

Primeiro, selecionamos o banco de dados dos tweets de teste:

In [65]:
teste_relevancia = teste.Teste
for i in teste_relevancia:
    calcula_prob(i)

extremamente relevante
extremamente relevante
extremamente irrelevante
extremamente relevante
extremamente irrelevante
extremamente relevante
extremamente relevante
extremamente relevante
extremamente relevante
extremamente relevante
extremamente relevante
extremamente relevante
relevante
extremamente relevante
extremamente relevante
extremamente irrelevante
extremamente relevante
extremamente irrelevante
extremamente relevante
extremamente relevante
extremamente relevante
extremamente relevante
extremamente relevante
extremamente relevante
extremamente irrelevante
extremamente relevante
extremamente relevante
relevante
extremamente relevante
extremamente irrelevante
extremamente relevante
extremamente relevante
extremamente irrelevante
extremamente relevante
extremamente irrelevante
extremamente relevante
extremamente irrelevante
extremamente relevante
extremamente relevante
extremamente relevante
extremamente irrelevante
extremamente relevante
extremamente relevante
extremamente rele

Analisamos o número de tweets relevantes: 168

Também analisamos o número de tweets irrelevantes: 32

Fizemos a contagem de quantos tweets o nosso contador acertou quanto a relevancia deles e os que ele nao acertou 

A partir disso, verificamos as porcentagens de cada um e chegamos a conclusao de que existe:

- 4% de positivos falsos

- 75,5% de positivos verdadeiros

- 12% de negativos verdadeiros

- 8,5% de negativos falsos


Ou seja, com isso concluimos que nosso programa foi capaz de acertar 87,5% dos casos, com um percentual de erro de apenas 12,5%.

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


Com base em todo o processo, podemos concluir que foi posível elaborar um classificador funcional, mas com espaço para melhorias. Isso se deve ao fato de uma parcela dos dados considerados como irrelevantes ser vista como relevante, demonstrando uma tendência do código de classificar uma frase dessa forma, pois o mesmo erro não foi visto com a mesma intensidade para o outro caso. Já no que tange à dupla negação e ao sarcasmo, vale ressaltar que o classificador trata essas situações como se fossem iguais às comuns. Contudo, isso não deve influenciar tanto o resultado, já que essas linguagens de expressão são utilizadas principalmente para avaliar um determinado assunto, enquanto o programa procura definir se o texto se trata desse tema ou não.
Dentre as possíveis melhorias, a mais promissora é a implementação de "ngramas". Isso se trata de, ao invés de separar cada palavra do texto avaliado e tratá-las de forma independente, considerar o texto como uma única palavra. Assim, a função compararia o texto com as palávras contidas em um dicionário e distribuidas em diversos ngramas. Dessa forma, se o texto inserido contiver duas palavras, o conteúdo do dicionário será dividido em bigramas. Para 3 palavras, haverá uma divisão de trigramas. Contudo, seria necessário possuir uma database mais extensa, sendo imprático realizar a categorização de cada tweet manualmente. 
Dentre as utilidades do projeto fora do tema observado (Fortnite), uma delas seria avaliar um conjunto de tweets e determinar se seu tema é o desajado pela empresa, como um de seus produtos. Dessa maneira, seria possível realizar um feedback e melhorar a performance da empresa no mercado. Contudo, devemos ressaltar que, no momento, o programa não coleta o seu próprio banco de dados, somente avaliando um que é dado previamente.
Com base no exposto acima, podemos dizer que o projeto merece receber maiores investimentos, já que mostrou ser eficaz até o dado desenvolvimento e pode ser ainda mais aperfeiçoado, além de que o sistema possui boas aplicações mercadológicas.

